# LEO Satellite Link Budget Example

This notebook demonstrates how to create a complete link budget analysis for a Low Earth Orbit (LEO) satellite communication system using the PyRadio library. We'll analyze both:

1. **Forward Link (Uplink)**: Ground station to satellite at 2.075 GHz
2. **Return Link (Downlink)**: Satellite to ground station at 2.25 GHz

The parameters used in this example are based on typical values for small satellite communications.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special

# Import PyRadio modules
from spacelink.antenna import Dish, FixedGain
from spacelink.mode import Mode
from spacelink.link import Link
from spacelink.cascade import Cascade, Stage
from spacelink.units import Q_, Hz, K, m, km, GHz, MHz, kHz, dB, dBW, dBm, dimensionless

# Spacecraft Setup

In [2]:
s0 = Stage(label="input cable", loss=Q_(0.5, dB), noise_temp=Q_(200, K))
s1 = Stage(label="LNA", gain=Q_(20, dB), noise_figure=Q_(0.5, dB))
s2 = Stage(label="cable 2", loss=Q_(0.5, dB), noise_temp=Q_(200, K))

receiver_front_end = Cascade([s0, s1, s2])

## Define Common Parameters

First, let's define some common parameters that will be used for both uplink and downlink calculations.

In [3]:
# Common parameters
distance = Q_(2000, 'km')  # Typical LEO satellite distance

# Create a function to return distance (allows for more complex models later)
def distance_function():
    return distance

gs_antenna = Dish(Q_(3.7, 'm'), 0.65, axial_ratio=3.0)


# Required Eb/N0 for reliable communications
required_ebno = 9.6  # dB (typical for BPSK with BER 10^-6)

## Define Forward Link (Uplink) Parameters

Now let's define the parameters specific to the uplink from the ground station to the satellite.

In [4]:
# uplink mode

uplink_mode = Mode(
    name="BPSK",
    coding_scheme="uncoded",
    modulation="BPSK",
    bits_per_symbol=Q_(1, dimensionless),
    code_rate=1.0,
    spectral_efficiency=0.5,
    required_ebno=9.6,
    implementation_loss=0.5,
)
# Create the uplink object
uplink = Link(
    frequency=Q_(2.075, 'GHz'),
    symbol_rate=Q_(2, kHz),
    tx_antenna=gs_antenna,
    rx_antenna=FixedGain(-20.0, axial_ratio=3.0),
    tx_power=Q_(100.0, 'W'),
    tx_front_end=Cascade(),
    rx_front_end=receiver_front_end,
    rx_antenna_noise_temp=Q_(200.0, 'K'),
    distance_fn=distance_function,
    mode = uplink_mode
)

## Analyze Forward Link (Uplink) Budget

In [5]:
# Display the uplink budget
print("FORWARD LINK (UPLINK) BUDGET:")
print(uplink)

# Show specific parameters
print(f"\nLink Details:")
print(f"Frequency: {uplink.frequency.to('MHz'):.2f}")
print(f"Distance: {distance.to('km'):.2f}")
print(f"Bandwidth: {uplink.bandwidth.to('kHz'):.2f}")
print(f"\nGround Station TX:")
print(f"Transmit Power: {uplink.tx_power.to('dBW'):.2f} dBW")
print(f"Dish Diameter: {gs_antenna.diameter.to('m'):.2f}")
print(f"Antenna Gain: {gs_antenna.gain(uplink.frequency):.2f} dB")
print(f"\nSatellite RX:")
print(f"Antenna Gain: {uplink.rx_antenna.gain(uplink.frequency):.2f} dB")
print(f"System Noise Temperature: {uplink.system_noise_temperature.to('K'):.2f}")

# Check if the link has sufficient margin
if uplink.margin > 3.0:
    print(f"\n✅ Forward link has adequate margin ({uplink.margin:.2f} dB)")
else:
    print(f"\n❌ Forward link may have insufficient margin ({uplink.margin:.2f} dB)")

FORWARD LINK (UPLINK) BUDGET:
Link Budget:
  EIRP: 56.2 dBW
  Path Loss: 164.8 dB
  Polarization Loss: 0.5 dB
  Received Power: -129.1 dBW
  System Noise Temperature: 441.9 K
  Noise Power: -166.1 dBW
  C/N: 37.0 dB
  Margin: 26.9 dB

Link Details:
Frequency: 2075.00 megahertz
Distance: 2000.00 kilometer
Bandwidth: 4.00 kilohertz

Ground Station TX:
Transmit Power: 20.00 decibelwatt dBW
Dish Diameter: 3.70 meter
Antenna Gain: 36.24 dB

Satellite RX:
Antenna Gain: -20.00 dB
System Noise Temperature: 441.95 kelvin

✅ Forward link has adequate margin (26.95 dB)


## Define Return Link (Downlink) Parameters

Now let's define the parameters specific to the downlink from the satellite to the ground station.

In [6]:
downlink_mode = Mode(
    name="BPSK",
    coding_scheme="uncoded",
    modulation="BPSK",
    bits_per_symbol=Q_(1, dimensionless),
    code_rate=1.0,
    spectral_efficiency=0.5,
    required_ebno=9.6,
    implementation_loss=0.5,
)
# Create the downlink object
downlink = Link(
    frequency=Q_(2.25, 'GHz'),
    symbol_rate=Q_(2, MHz),
    tx_antenna=FixedGain(5.0, axial_ratio=3.0),
    rx_antenna=gs_antenna,
    tx_power=Q_(5.0, 'W'),
    tx_front_end=Cascade(),
    rx_front_end=Cascade([Stage(label='rx_fe', noise_temp=Q_(100.0, 'K'))]),
    rx_antenna_noise_temp=Q_(50.0, 'K'),
    distance_fn=distance_function,
    mode = downlink_mode
)

## Analyze Return Link (Downlink) Budget

In [7]:
# Display the downlink budget
print("RETURN LINK (DOWNLINK) BUDGET:")
print(downlink)

# Show specific parameters
print(f"\nLink Details:")
print(f"Frequency: {downlink.frequency.to('MHz'):.2f}")
print(f"Distance: {downlink.distance.to('km'):.2f}")
print(f"Bandwidth: {downlink.bandwidth.to('MHz'):.2f}")
print(f"\nSatellite TX:")
print(f"Transmit Power: {downlink.tx_power.to('dBW'):.2f} dBW")
print(f"Antenna Gain: {downlink.tx_antenna.gain(downlink.frequency):.2f} dB")
print(f"\nGround Station RX:")
print(f"Dish Diameter: {gs_antenna.diameter.to('m'):.2f}")
print(f"Antenna Gain: {gs_antenna.gain(downlink.frequency):.2f} dB")
print(f"System Noise Temperature: {downlink.system_noise_temperature.to('K'):.2f}")

# Check if the link has sufficient margin
if downlink.margin > 3.0:
    print(f"\n✅ Return link has adequate margin ({downlink.margin:.2f} dB)")
else:
    print(f"\n❌ Return link may have insufficient margin ({downlink.margin:.2f} dB)")

RETURN LINK (DOWNLINK) BUDGET:
Link Budget:
  EIRP: 12.0 dBW
  Path Loss: 165.5 dB
  Polarization Loss: 0.5 dB
  Received Power: -117.1 dBW
  System Noise Temperature: 150.0 K
  Noise Power: -140.8 dBW
  C/N: 23.7 dB
  Margin: 13.6 dB

Link Details:
Frequency: 2250.00 megahertz
Distance: 2000.00 kilometer


AttributeError: 'Mode' object has no attribute 'bandwidth'

## Link Margin vs. Distance Analysis

Let's analyze how the link margin varies with distance for both uplink and downlink.

In [ ]:
# Define a range of distances
distances = np.linspace(500, 3000, 50)  # km
uplink_margins = []
downlink_margins = []

# Calculate margins at each distance
for d in distances:
    # Update the distance
    distance = Q_(d, 'km')
    
    # Calculate margins
    uplink_margins.append(uplink.margin)
    downlink_margins.append(downlink.margin)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(distances, uplink_margins, 'b-', label='Forward Link (Uplink)')
plt.plot(distances, downlink_margins, 'r-', label='Return Link (Downlink)')
plt.grid(True)
plt.xlabel('Distance (km)')
plt.ylabel('Link Margin (dB)')
plt.title('Link Margin vs. Distance')
plt.axhline(y=3, color='gray', linestyle='--', label='3 dB Minimum Margin')
plt.axvline(x=2000, color='green', linestyle='--', label='Nominal Distance (2000 km)')
plt.legend()
plt.show()

## Conclusion

This notebook demonstrated how to use the PyRadio library to analyze both uplink and downlink budgets for a LEO satellite communication system. Key findings include:

1. The forward link (uplink) has a substantial margin due to the high-power ground station transmitter and large dish antenna.
2. The return link (downlink) has a smaller but still adequate margin even with a much lower transmit power from the satellite.
3. The link margin decreases with increasing distance, but both links maintain adequate performance over the expected distance range of the LEO satellite.
4. The BER analysis confirms that both links exceed the required Eb/N0 for reliable communications with a BPSK modulation scheme.

This analysis provides confidence that the communication system design will meet the requirements for reliable satellite communications.